In [23]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import xgboost

from functions import make_dataset

from functions import make_portfolio

from sklearn.utils import shuffle

import time

import pickle

In [24]:
features2 = pd.read_pickle("training_sets/features2")

In [25]:
Date2 = np.load("training_sets/Date2.npy", allow_pickle = True)

In [26]:
Tickers2 = np.load("training_sets/Tickers2.npy", allow_pickle = True)

In [27]:
with open('Training_window_test/top_50.pkl', 'rb') as f:
    top_50 = pickle.load(f)

In [28]:
cols = np.random.choice(top_50, 15, replace = False)

In [29]:
cols

array(['EV/FCF', 'Cash & Cash Equivalents/Total Liabilities_QC',
       'Total Current Liabilities/Total Liabilities_QC',
       'Working Capital_QC', 'BOL_indicator_10', 'MFI_40',
       'close_120/Close_160', 'Revenue', 'CMO_160', 'ADX_1_11',
       'Total Equity_YC', 'Retained Earnings/Total Liabilities_YC',
       'ABAND_indicator_10', 'Total Equity', 'AROON_120'], dtype='<U58')

In [30]:
def make_dataset_eval_with_extra_return_40_term(df, training_window, prediction_window, eval_size, prediction_date_number, Date_array, cols, pred_col):
    
    training_period = Date_array[prediction_date_number - training_window - prediction_window + 1: prediction_date_number - prediction_window +1]

    prediction_period = Date_array[prediction_date_number :prediction_date_number + eval_size + 1]
    
    X_train = shuffle(df.loc[training_period, np.append(cols, pred_col)])
    X_train = X_train.replace([np.inf, -np.inf], np.NaN)
    X_train = X_train.dropna(axis = 0)

    Y_train = X_train.pop(pred_col)
    
    
    
    X_test = shuffle(df.loc[prediction_period, np.append(np.append(cols, pred_col), "return_40")])
    X_test = X_test.replace([np.inf, -np.inf], np.NaN)
    X_test = X_test.dropna(axis = 0)
    
    Y_test = X_test.pop(pred_col)
    Y_return_40 = X_test.pop("return_40")
    
    return X_train, X_test, Y_train, Y_test, Y_return_40

In [31]:
params1 = {
  'colsample_bynode': 0.6,
    'colsample_bytree' : 0.6,
    'colsample_bylevel' : 0.5,
  'learning_rate': 0.3,
  'max_depth': 3,
  'num_parallel_tree': 100,
  'objective': 'multi:softprob',
  'subsample': 0.9,
'num_class' : 2,
  'tree_method': 'gpu_hist',
    "verbosity" : 0,
    "lambda" : 1,
    "alpha" : 0,
    "eval_metric": "mlogloss"
}

In [32]:
X_train, X_eval, Y_train, Y_eval, Y_eval_return_40 = make_dataset_eval_with_extra_return_40_term(features2, 200, 40, 35, 375, Date2, cols, "Class_binary_40_20")

In [33]:
train_weights = Y_train.map({0:19, 1:1})


In [34]:
xgtrain = xgboost.DMatrix(X_train.values, Y_train.values, feature_names = cols, weight = train_weights)
xgeval = xgboost.DMatrix(X_eval.values, Y_eval.values, feature_names = cols)


In [35]:
a = {}

In [128]:
def return_metric(predt):
    
    try:
        
        return np.mean(Y_eval_return_40[np.argsort(predt[:,0])[::-1][:12]])
    
    except:
        return  float(1)

In [129]:
class my_callback(xgboost.callback.TrainingCallback):
    
    def __init__(self, data, a):
        self.data_mine = data
        self.results_mine = a
        
        
    def after_iteration(self, model, epoch, evals_log):
        
        
        predictions_mine = model.predict(self.data_mine)
        
        score_mine = return_metric(predictions_mine)
        self.results_mine.append(score_mine) 
    
        return False
        
        

        

In [130]:
a = []

In [131]:
mycallback = my_callback(xgeval, a)

In [134]:
start_time = time.perf_counter()
a = []
mycallback = my_callback(xgeval, a)
bst = xgboost.train(params1, xgtrain, num_boost_round =50, verbose_eval = False, callbacks = [mycallback])
print(time.perf_counter() - start_time)

18.22659349999958
